In [1]:
!pip install google-cloud-bigquery

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/227.8 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/227.8 kB 1.3 MB/s eta 0:00:01
   ---------------------------- ----------- 163.8/227.8 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 227.8/227.8 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/80.6 kB ? eta -:--:--
   ---------------------------------------- 80.6/80.6 kB 4.4 MB/s eta 0:00:00
  Created wheel for google-crc32c: filename=google_crc32c-1.5.0-py3-none-any.whl size=13046 sha256=8261300c084dcf54cf5d3a692ae3ded51ec237554af03dcad15946ce05890199
  Stored in directory: c:\users\rames\appdata\lo

In [13]:
!pip install --upgrade google-cloud-bigquery[all]

  Using cached ipywidgets-8.1.1-py3-none-any.whl.metadata (2.4 kB)
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     --------------------------------- ---- 51.2/57.6 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 607.3 kB/s eta 0:00:00
     ---------------------------------------- 0.0/51.1 kB ? eta -:--:--
     ---------------------------------------- 51.1/51.1 kB 1.3 MB/s eta 0:00:00
  Using cached widgetsnbextension-4.0.9-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.9-py3-none-any.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ----------- ---------------------------- 0.3/1.1 MB 10.2 MB/s eta 0:00:01
   --------------------------- ------------ 0.7/1.1 MB 9.5 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 10.0 MB/s eta 0:00:00
   ------------------------

In [18]:
!pip install pyarrow

#The pandas-gbq library provides a simple interface for running queries and uploading pandas dataframes to BigQuery. 
#It is a thin wrapper around the BigQuery client library, google-cloud-bigquery.

In [56]:
!pip install --upgrade pandas-gbq google-cloud-bigquery[bqstorage,pandas]

     ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
     ------- ------------------------------- 30.7/151.7 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 151.7/151.7 kB 1.8 MB/s eta 0:00:00


In [58]:
!pip install db-dtypes 

In [1]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./key.json"

In [2]:
from google.cloud import bigquery 
client = bigquery.Client()

In [4]:
#using Panda BigQuery Library (pandas-gbq)
import pandas

sql = """
    SELECT fare, trip_miles, trip_seconds FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` LIMIT 5
"""

# Run a Standard SQL query using the environment's default project
df = pandas.read_gbq(sql, dialect="standard")

# Run a Standard SQL query with the project set explicitly
#project_id = "your-project-id"
#df = pandas.read_gbq(sql, project_id=project_id, dialect="standard")
df = pandas.read_gbq(sql,  dialect="standard")

In [5]:
df

,fare,trip_miles,trip_seconds
0,3.50,0.0,57
1,27.00,0.0,8
2,3.25,0.0,0
3,26.50,0.0,2100
4,30.00,0.0,25


In [6]:
# Using Google client library
# BigQuery first returns a query object and from there you can get the result using .result(). 
# But that is an Iterator object so you have to iterate to get the values. 
QUERY = ( 'SELECT fare, trip_miles, trip_seconds FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` where trip_miles > 0  LIMIT 5' )
query_job = client.query(QUERY)
rows = query_job.result()
for row in rows:
    print(row.fare , row.trip_miles, row.trip_seconds)

5.05 0.1 180
4.85 0.1 360
4.45 0.1 180
16.05 0.8 1320
5.45 0.2 180


In [68]:
type(row)

google.cloud.bigquery.table.Row

In [ ]:
project = "bigquery-public-data"
dataset_id = "chicago_taxi_trips"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("taxi_trips")
table = client.get_table(table_ref)

df = client.list_rows(table).to_dataframe()
df.head(5)

In [9]:
# Using Google client library
from google.api_core.exceptions import InternalServerError
from google.api_core.exceptions import ServiceUnavailable
from google.api_core.exceptions import TooManyRequests
from google.cloud import bigquery
#from google.cloud import storage
import pandas
sql = ( 'SELECT fare, trip_miles, trip_seconds FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` where trip_miles > 0  LIMIT 5;' )
df = client.query(sql).to_dataframe()
df.head()

,fare,trip_miles,trip_seconds
0,5.05,0.1,180
1,4.85,0.1,360
2,4.45,0.1,180
3,16.05,0.8,1320
4,5.45,0.2,180


In [ ]:
# Count Rows where Fare Is Not Null

sql = ( 'SELECT COUNT(fare) AS count_of_non_null_fares FROM bigquery-public-data.chicago_taxi_trips.taxi_trips WHERE fare IS NOT NULL;' )
df = client.query(sql).to_dataframe()
df.head(5)

In [ ]:
# Find Correlations between Fare and Other Columns

sql = ( 'SELECT AVG(fare) AS AVG_FARE FROM bigquery-public-data.chicago_taxi_trips.taxi_trips LIMIT 10000;')
df = client.query(sql).to_dataframe()
df.head(5)

In [ ]:
# Find Average Fare

sql - ('SELECT EXTRACT(DAYOFWEEK FROM trip_start_timestamp) AS day_of_week, AVG(fare) AS average_fare FROM bigquery-public-data.chicago_taxi_trips.taxi_trips WHERE fare IS NOT NULL GROUP BY day_of_week ORDER BY day_of_week;')
df = client.query(sql).to_dataframe()
df.head(10)